In [ ]:
#pip install awswrangler

In [34]:
import awswrangler as wr

Create Governed Table(metadata only)

In [35]:
wr.catalog.create_parquet_table(
    database='governed_demo',
    table='governed_demo_table',
    path='s3://governedtable-seba-oregon/test/',
    columns_types={'registration_dttm':'Timestamp',
                  'id':'int',
                  'first_name':'String',
                  'last_name':'String',
                  'email':'String',
                  'gender':'String',
                  'ip_address':'String',
                  'cc':'String',
                  'country':'String',
                  'birthdate':'String',
                  'salary':'double',
                  'title':'string',
                  'comment':'string'},
    compression='snappy',
    description='governed_table example',
    table_type='GOVERNED'
)

In [ ]:
#해당 실행 전에 S3등록하고 admin 유저 등록 권한 등록을 수행하여야 함

Read existing csv data from S3 into dataframe

In [14]:
df=wr.s3.read_parquet(r's3://governedtable-seba-oregon/raw/')
df.head()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,


Filter Select records for insert into governed table

In [15]:
filtered_df=df[df.gender.isin(['Female' ])]
filtered_df.head()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
5,2016-02-03 07:22:34,6,Kathryn,White,kwhite5@google.com,Female,195.131.81.179,3583136326049310,Indonesia,2/25/1983,69227.11,Account Executive,
9,2016-02-03 18:29:47,10,Emily,Stewart,estewart9@opensource.org,Female,143.28.251.245,3574254110301671,Nigeria,1/28/1997,27234.28,Health Coach IV,


create a new glue transaction

In [16]:
transaction_1 = wr.lakeformation.start_transaction(read_only=False)
transaction_1

'9ca05e0754314fa2b453d4b18c4abcaf'

write data to S3

In [18]:
wr.s3.to_parquet(
    df=filtered_df,
    path='s3://governedtable-seba-oregon/test/',
    dataset=True,
    compression='snappy',
    database='governed_demo',
    table='governed_demo_table',
    parameters={"num_cols":str(len(df.columns)),"num_rows":str(len(df.index))},
    mode='overwrite',
    table_type='GOVERNED',
    transaction_id=transaction_1)

{'paths': ['s3://governedtable-seba-oregon/test/292484da245e445da51d1a51ca082ca5.snappy.parquet'],
 'partitions_values': {}}

Commit Lake formation Transaction

In [21]:
wr.lakeformation.commit_transaction(transaction_1)
transaction = wr.lakeformation.describe_transaction(transaction_1)
transaction


'committed'

Insert Additional data into governed table

In [22]:
filtered_df2 = df[df.gender.isin(['Female' ])]

In [23]:
transaction_2 = wr.lakeformation.start_transaction(read_only=False)

In [24]:
wr.s3.to_parquet(
    df=filtered_df2,
    path='s3://governedtable-seba-oregon/test/',
    dataset=True,
    compression='snappy',
    database='governed_demo',
    table='governed_demo_table',
    parameters={"num_cols":str(len(df.columns)),"num_rows":str(len(df.index))},
    mode='append',
    table_type='GOVERNED',
    transaction_id=transaction_2)

{'paths': ['s3://governedtable-seba-oregon/test/bd26faa684024e448b0f0929708e3b11.snappy.parquet'],
 'partitions_values': {}}

In [28]:
wr.lakeformation.commit_transaction(transaction_2)
transaction = wr.lakeformation.describe_transaction(transaction_2)
transaction

'committed'

Read from govenred table

In [29]:
df3 = wr.lakeformation.read_sql_query(
    sql=f"SELECT * FROM {'governed_demo_table'};",
    database='governed_demo')
df3.head()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
2,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
3,2016-02-03 07:22:34,6,Kathryn,White,kwhite5@google.com,Female,195.131.81.179,3583136326049310,Indonesia,2/25/1983,69227.11,Account Executive,
4,2016-02-03 18:29:47,10,Emily,Stewart,estewart9@opensource.org,Female,143.28.251.245,3574254110301671,Nigeria,1/28/1997,27234.28,Health Coach IV,


In [31]:
df3.shape[0]

964

Read from governed table transaction

In [33]:
df4 = wr.lakeformation.read_sql_query(
    sql=f"SELECT * FROM {'governed_demo_table'};",
    database='governed_demo',
    transaction_id=transaction_1)
df4.shape[0]

482

In [48]:
!pip install awswrangler

In [45]:
import awswrangler as wr
import time

In [47]:
start = time.time()
query_result = wr.athena.read_sql_query(
    sql="select * from dorei_test_db.raw",
    database='dorei_test_db'
)
end = time.time()
print(f"{end - start:.5f} sec")

4.44096 sec
